# distance and direciton

Let try to opt for distance and direction with

$L1loss(y_1-y_0, y_{true})$

where $y_1=model(x_1)$

So I'm optimising for the hidden states to be the correct distance and direcioton away. It's like the margin raning loss.


links:
- [loading](https://github.com/deep-diver/LLM-As-Chatbot/blob/main/models/alpaca.py)
- [dict](https://github.com/deep-diver/LLM-As-Chatbot/blob/c79e855a492a968b54bac223e66dc9db448d6eba/model_cards.json#L143)
- [prompt_format](https://github.com/deep-diver/PingPong/blob/main/src/pingpong/alpaca.py)

In [1]:
# import your package
%load_ext autoreload
%autoreload 2


In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
plt.style.use('ggplot')

from typing import Optional, List, Dict, Union

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor
from torch import optim
from torch.utils.data import random_split, DataLoader, TensorDataset
from src.helpers.ds import shuffle_dataset_by
from pathlib import Path

import transformers

import lightning.pytorch as pl
# from dataclasses import dataclass

# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
# from sklearn.preprocessing import RobustScaler

from tqdm.auto import tqdm
import os

from loguru import logger
logger.add(os.sys.stderr, format="{time} {level} {message}", level="INFO")



transformers.__version__


/media/wassname/SGIronWolf/projects5/elk/discovering_latent_knowledge/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'4.34.1'

In [3]:
from src.helpers.lightning import read_metrics_csv


# Datasets



In [4]:
from datasets import load_from_disk, concatenate_datasets
from src.datasets.load import ds2df, load_ds

feats = ['hidden_states', 'head_activation_and_grad', 'mlp_activation_and_grad', 'residual_stream', 'w_grads_attn', 'w_grads_mlp', 'hidden_states2', 'residual_stream2', ]

fs = [
    # "../.ds/TheBloke_WizardCoder-Python-13B-V1.0-GPTQ_amazon_polarity_test_615",
    "../.ds/TheBloke_WizardCoder-Python-13B-V1.0-GPTQ_amazon_polarity_train_555",
    # "../.ds/TheBloke_WizardCoder-Python-13B-V1.0-GPTQ_glue_qnli_test_615",    
    "../.ds/TheBloke_WizardCoder-Python-13B-V1.0-GPTQ_glue_qnli_train_555",
    # "../.ds/TheBloke_WizardCoder-Python-13B-V1.0-GPTQ_super_glue_boolq_test_615",
    "../.ds/TheBloke_WizardCoder-Python-13B-V1.0-GPTQ_super_glue_boolq_train_555",
]

# dss = [load_from_disk(f) for f in fs]


In [5]:
d = load_from_disk(fs[-1])
ds = d.select(range(10))


In [6]:
h = ds['end_hidden_states']
np.array(h).shape


(10, 41, 5120, 2)

In [7]:

ds = ds.with_format("numpy")
h = ds['end_hidden_states']
h.shape


(10, 41, 5120, 2)

In [8]:
ds = ds.with_format("torch")
h = ds['end_hidden_states']
h.shape


torch.Size([10, 41, 5120, 2])

In [47]:
h.diff(1)


torch.Size([10, 41, 5120, 2])

In [9]:
ds.features


{'end_hidden_states': Sequence(feature=Sequence(feature=Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None), length=-1, id=None), length=-1, id=None),
 'end_logits': Sequence(feature=Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None), length=-1, id=None),
 'instructed_to_lie': Value(dtype='bool', id=None),
 'question': Value(dtype='string', id=None),
 'answer_choices': Sequence(feature=Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), length=-1, id=None),
 'choice_ids': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None),
 'template_name': Value(dtype='string', id=None),
 'sys_instr_name': Value(dtype='string', id=None),
 'example_i': Value(dtype='int64', id=None),
 'label_true': Value(dtype='int64', id=None),
 'input_truncated': Value(dtype='string', id=None),
 'truncated': Value(dtype='float64', id=None),
 'text_ans': Value(dtype='string', id=None),
 'add_ans': Sequence(feat

In [33]:
from ctypes import Array
import numpy as np
from datasets.features import Sequence, Value, Features, Array2D, Array3D, Features

# from datasets import batch

ds['end_hidden_states'].shape
# ds.map(lambda x: {'end_hidden_states': x['end_hidden_states'] }, features=Array2D(ds['end_hidden_states'].shape, dtype=np.float16), batched=True, batch_size=128)

ds.map(lambda x: {'end_hidden_states': x['end_hidden_states'] }, features=Features({'end_hidden_states': Array3D(shape=ds['end_hidden_states'].shape[1:], dtype=np.float16)}), batched=True, batch_size=5)


Map:   0%|          | 0/10 [00:08<?, ? examples/s]


TypeError: unsupported operand type(s) for +: 'type' and 'str'

In [36]:
x = ds[:]


In [37]:
from re import A
from datasets.features import Sequence, Value, Features, Array2D, Array3D, Features

features = {
    "end_hidden_states": Array3D(dtype="float16", id=None, shape=x['end_hidden_states'].shape[1:]),
    "end_logits": Array2D(dtype="float16", id=None, shape=x['end_logits'].shape[1:]),
    "add_ans": Array2D(dtype="float16", id=None, shape=x['add_ans'].shape[1:]),
    "label_true": Value(dtype="int64", id=None),
    "instructed_to_lie": Value(dtype="bool", id=None),
    "question": Value(dtype="string", id=None),
    "answer_choices": Sequence(
        feature=Sequence(feature=Value(dtype="string", id=None), length=-1, id=None),
        length=-1,
        id=None,
    ),
    "choice_ids": Sequence(
        feature=Sequence(feature=Value(dtype="int64", id=None), length=-1, id=None),
        length=-1,
        id=None,
    ),
    "template_name": Value(dtype="string", id=None),
    "sys_instr_name": Value(dtype="string", id=None),
    "example_i": Value(dtype="int64", id=None),
    "input_truncated": Value(dtype="string", id=None),
    "truncated": Value(dtype="float64", id=None),
    "text_ans": Value(dtype="string", id=None),
    "ans": Sequence(feature=Value(dtype="float16", id=None), length=-1, id=None),
}
features = Features(features)
features


{'end_hidden_states': Array3D(shape=(41, 5120, 2), dtype='float16', id=None),
 'end_logits': Array2D(shape=(32001, 2), dtype='float16', id=None),
 'add_ans': Array2D(shape=(2, 2), dtype='float16', id=None),
 'label_true': Value(dtype='int64', id=None),
 'instructed_to_lie': Value(dtype='bool', id=None),
 'question': Value(dtype='string', id=None),
 'answer_choices': Sequence(feature=Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), length=-1, id=None),
 'choice_ids': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None),
 'template_name': Value(dtype='string', id=None),
 'sys_instr_name': Value(dtype='string', id=None),
 'example_i': Value(dtype='int64', id=None),
 'input_truncated': Value(dtype='string', id=None),
 'truncated': Value(dtype='float64', id=None),
 'text_ans': Value(dtype='string', id=None),
 'ans': Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None)}

In [38]:
from datasets import Dataset
dd = Dataset.from_dict(x, features=features)
dd.features


ArrowNotImplementedError: Unsupported cast from float to halffloat using function cast_half_float

In [44]:
x = ds[:]

x['end_hidden_states'] = x['end_hidden_states'].numpy().astype(np.float16)
x['end_hidden_states'].dtype


array([[[[-5.2977e-04, -5.2977e-04],
         [-8.4114e-04, -8.4114e-04],
         [-3.4213e-04, -3.4213e-04],
         ...,
         [ 1.7271e-03,  1.7271e-03],
         [ 2.6798e-04,  2.6798e-04],
         [-1.9276e-04, -1.9276e-04]],

        [[ 4.2725e-04,  4.2725e-04],
         [-1.6918e-03, -1.6918e-03],
         [-2.5772e-02, -2.5772e-02],
         ...,
         [ 8.3008e-03,  8.3008e-03],
         [ 6.3744e-03,  6.3744e-03],
         [-4.1733e-03, -4.1733e-03]],

        [[ 2.3102e-02,  2.3102e-02],
         [-2.2369e-02, -2.2369e-02],
         [-6.0059e-02, -6.0059e-02],
         ...,
         [ 3.1067e-02,  3.1067e-02],
         [-1.2634e-02, -1.2634e-02],
         [ 1.4992e-02,  1.4992e-02]],

        ...,

        [[-1.6367e+00, -1.2021e+00],
         [-1.6357e+00, -3.7158e-01],
         [-3.5527e+00, -7.9570e+00],
         ...,
         [-2.7363e+00, -5.6641e+00],
         [-1.1875e+00,  5.5586e+00],
         [ 4.3066e-01,  4.9883e+00]],

        [[-1.4160e+00, -2.4707e+00

In [42]:
dd = Dataset.from_dict(x)
dd.features


{'end_hidden_states': Sequence(feature=Sequence(feature=Sequence(feature=Value(dtype='float16', id=None), length=-1, id=None), length=-1, id=None), length=-1, id=None),
 'end_logits': Sequence(feature=Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None), length=-1, id=None),
 'instructed_to_lie': Value(dtype='bool', id=None),
 'question': Value(dtype='string', id=None),
 'answer_choices': Sequence(feature=Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), length=-1, id=None),
 'choice_ids': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None),
 'template_name': Value(dtype='string', id=None),
 'sys_instr_name': Value(dtype='string', id=None),
 'example_i': Value(dtype='int64', id=None),
 'label_true': Value(dtype='int64', id=None),
 'input_truncated': Value(dtype='string', id=None),
 'truncated': Value(dtype='float32', id=None),
 'text_ans': Value(dtype='string', id=None),
 'add_ans': Sequence(feat